In [9]:
# Добавляем нужные библиотеки
from sklearn.model_selection import train_test_split
import pandas as pd
import matplotlib.pyplot as plt
import numpy as np
import tensorflow as tf
import sklearn
from sklearn.metrics import f1_score, roc_auc_score
from sklearn.preprocessing import LabelEncoder
from sklearn.ensemble import GradientBoostingClassifier
from sklearn.preprocessing import StandardScaler
from sklearn.decomposition import PCA
from sklearn.model_selection import GridSearchCV
from sklearn.svm import SVC
from sklearn.preprocessing import StandardScaler, OneHotEncoder
from sklearn.compose import ColumnTransformer
from sklearn.pipeline import Pipeline


In [10]:
# загружаем таблицы данных
members_pd = pd.read_csv("members.csv")
songs_pd = pd.read_csv('songs.csv')
sample_pd = pd.read_csv("sample_submission.csv")
tranin_data_pd = pd.read_csv('train.csv')

In [11]:
#Соеденяем все таблицы в одну
merge_df = pd.merge(tranin_data_pd, members_pd, on="msno", how='inner')
train_df = pd.merge(merge_df, songs_pd, on="song_id", how='inner')

In [14]:
# Разбиваем датасет на фичи и таргетные значения
Y_data = train_df["target"]
X_data = train_df.drop(columns="target",axis= 1)
X_data.fillna("0", inplace=True)
# выделяем числовые данные и категориальные данные
numeric_features = ['city', 'bd', 'registered_via', "expiration_date", 'song_length','language']  
categorical_features = ['msno', 'song_id','source_system_tab', 'source_screen_name','source_type','gender', 'artist_name',  'composer', 'genre_ids','lyricist'] 

X_processed = X_data.copy()

# Применяем Label Encoding к каждому категориальному признаку, чтоб перевести его в числовой
# Автоматически алгоритмы из sklearn с этим не справляются
label_encoder = LabelEncoder()
for feature in categorical_features:
    X_processed[feature] = label_encoder.fit_transform(X_data[feature])

/tmp/ipykernel_29049/2184497842.py:4: FutureWarning: Setting an item of incompatible dtype is deprecated and will raise in a future error of pandas. Value '0' has dtype incompatible with float64, please explicitly cast to a compatible dtype first.
  X_data.fillna("0", inplace=True)


In [15]:
# Разделяем выборку 
X_data = np.array(X_processed)
Y_data = np.array(Y_data)
x_train, x_test, y_train, y_test = train_test_split(X_data, Y_data, train_size=0.7)


In [16]:
# Попробуем использовать Градиентный бустинг. Он довольно хорошо справляется с данными, в которых много пробелов
# Так же случайный лес подходит для бинарной классификации
clf = sklearn.ensemble.HistGradientBoostingClassifier(max_depth = 32,l2_regularization = 0.2)
clf.fit(x_train, y_train)

HistGradientBoostingClassifier(l2_regularization=0.2, max_depth=32)

In [ ]:
y_pred = clf.predict(x_test)
ndcg = sklearn.metrics.ndcg_score(y_test, y_pred, k= 20)
print("NDCG@k -score:", ndcg)


F1-score: 0.719390658946051
ROC AUC: 0.6593907202519399


In [18]:
# Попробуем так же метод опорных векторов, ему по идее тоже не особо критичны пробелы в данных, так как вектора с пробелами просто не будут опорными

sv_machine = SVC().fit(x_train,y_train)

In [ ]:
y_pred = sv_machine.predict(x_test)
ndcg = sklearn.metrics.ndcg_score(y_test, y_pred, k= 20)
print("NDCG@k -score:", ndcg)


In [ ]:
#Попробуем еще радномный лес. Он тоже должен хорошо рабоать с пробелами в данных.
rf = sklearn.ensemble.RandomForestClassifier(max_depth=16, random_state=0)
rf.fit(x_train, y_train)

In [ ]:
y_pred = rf.predict(x_test)
ndcg = sklearn.metrics.ndcg_score(y_test, y_pred, k= 20)
print("NDCG@k -score:", ndcg)
